These notebooks are used to compare a base and scenario, from expanded surveys or model outputs, in H5 format. To run: from the menu bar above, choose **Cell -> Run All ** or run lines individually. Use the toggle button below to hide/show the raw Python code.

## School and Workplace Location

*Summaries:*
    - Workplace Location
    - School Location
---

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import os
import numpy as np
import pandas as pd
import h5py
import pylab as P
from IPython.display import display, display_pretty, Javascript, HTML
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt

# Show charts in notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
# Define data sources

# 2006 survey
survey06_dir = r'R:\SoundCast\releases\TransportationFutures2010\scripts\summarize'


# 2014 survey
survey14_dir = r'D:\travel-studies\2014\estimation'

In [5]:
# Read Model Scenario Results
scen = h5py.File(survey06_dir + r'/survey.h5','r+')
scen_name = '2006 Survey'

In [6]:
# Read Base Data
base_file = r'/survey14.h5'

base = h5py.File(survey14_dir + base_file ,'r+')
base_name = '2014 Survey'

In [8]:
def build_df(h5file, h5table, var_dict, survey_file=False):
    ''' Convert H5 into dataframe '''
    data = {}
    if survey_file:
        # survey h5 have nested data structure, different than daysim_outputs
        for col_name, var in var_dict.iteritems():
            data[col_name] = [i[0] for i in h5file[h5table][var][:]]
    else:
        for col_name, var in var_dict.iteritems():
            data[col_name] = [i for i in h5file[h5table][var][:]]

    return pd.DataFrame(data)

In [33]:
base['Person']['pno'][:][1]

(1,)

In [9]:
tripdict={'Household ID': 'hhno',
            'Person Number': 'pno',
            'Travel Time':'travtime',
            'Travel Cost': 'travcost',
            'Travel Distance': 'travdist',
            'Mode': 'mode',
            'Purpose':'dpurp',
            'Departure Time': 'deptm',
            'Expansion Factor': 'trexpfac'}

In [34]:
trip_scen = build_df(h5file=scen, h5table='Trip', var_dict=tripdict, survey_file=True)
trip_base = build_df(h5file=base, h5table='Trip', var_dict=tripdict, survey_file=True)

In [82]:
persondict={'Household ID': 'hhno',
            'Person Number': 'pno',
            'Transit Pass': 'ptpass',
            'Auto Time to Work': 'pwautime',
            'Auto Distance to Work': 'pwaudist',
            'Worker Type': 'pwtyp',
            'Student Type': 'pstyp',
            'Usual Commute Mode': 'puwmode',
            'Workplace TAZ': 'pwtaz',
            'School TAZ': 'pstaz',
            'Age': 'pagey',
            'Person Type': 'pptyp',
            'Expansion Factor': 'psexpfac'}

In [83]:
person_scen = build_df(h5file=scen, h5table='Person', var_dict=persondict, survey_file=True)
person_base = build_df(h5file=base, h5table='Person', var_dict=persondict, survey_file=True)

In [84]:
# Create unique ID for person by concatenating household ID and person number 
person_scen['personID'] = (person_scen['Household ID'].astype('str')+person_scen['Person Number'].astype('str')).astype('int')
person_base['personID'] = (person_base['Household ID'].astype('str')+person_base['Person Number'].astype('str')).astype('int')

In [85]:
hhdict={'Household ID': 'hhno',
        'Household Size': 'hhsize',
        'Household Vehicles': 'hhvehs',
        'Household Workers': 'hhwkrs',
        'Household Income': 'hhincome',
        'Household TAZ': 'hhtaz',
        'Expansion Factor': 'hhexpfac'}

In [86]:
hh_scen = build_df(h5file=scen, h5table='Household', var_dict=hhdict, survey_file=True)
hh_base = build_df(h5file=base, h5table='Household', var_dict=hhdict, survey_file=True)

In [87]:
# Add labels for worker type
labels = {
  0: "Not a worker",  
  1: "Full-time worker",
  2: "Part-time worker",
}

person_base['Worker Type'] = ([labels[x] for x in person_base['Worker Type']])
person_scen['Worker Type'] = ([labels[x] for x in person_scen['Worker Type']])

In [88]:
# Add district labels
labels = {
  0: "Not a worker",  
  1: "Full-time worker",
  2: "Part-time worker",
}

person_base['District'] = ([labels[x] for x in person_base['District']])
person_scen['District'] = ([labels[x] for x in person_scen['District']])

KeyError: 'District'

In [89]:
# # Add labels for student type
# labels = {
#   0: "Not a student",  
#   1: "Full-time student",
#   2: "Part-time student",
# }

# person_base['Student Type'] = ([labels[x] for x in person_base['Student Type']])
# person_scen['Student Type'] = ([labels[x] for x in person_scen['Student Type']])

In [90]:
# Join household records to person records
hh_per_scen = pd.merge(left=person_scen, right=hh_scen,on='Household ID',suffixes=('_p','_h'))
hh_per_base = pd.merge(left=person_base, right=hh_base,on='Household ID',suffixes=('_p','_h'))

In [91]:
# Join household geography
taz_geog = pd.read_csv(r'utils/taz_lookup.csv')
taz_geog.reindex
hh_per_scen_home_geog = pd.merge(hh_per_scen, taz_geog, left_on='Household TAZ', right_on='TAZ')
hh_per_base_home_geog = pd.merge(hh_per_base, taz_geog, left_on='Household TAZ', right_on='TAZ')

In [92]:
# Join workplace geography
hh_per_scen_work_geog = pd.merge(hh_per_scen, taz_geog, left_on='Workplace TAZ', right_on='TAZ')
hh_per_base_work_geog = pd.merge(hh_per_base, taz_geog, left_on='Workplace TAZ', right_on='TAZ')

---

## Jobs by Location

### Jobs by County

In [74]:
df = pd.DataFrame([hh_per_scen_work_geog.groupby('County').sum()['Expansion Factor_h'],
                   hh_per_base_work_geog.groupby('County').sum()['Expansion Factor_h']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
County,,
King,911361.10,1105040.6864
Kitsap,77365.82,79099.7800
Pierce,209718.71,254498.6217
Snohomish,198680.46,219190.8260


In [75]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

,2006 Survey,2014 Survey
County,,
King,0.652311,0.666559
Kitsap,0.055375,0.047713
Pierce,0.150107,0.153513
Snohomish,0.142207,0.132216


In [77]:
display_charts(df_new, kind='bar', title='Job Location Distribution by County')

### Jobs by District

In [78]:
district_col = 'New DistrictName'
df = pd.DataFrame([hh_per_scen_work_geog.groupby(district_col).sum()['Expansion Factor_h'],
                   hh_per_base_work_geog.groupby(district_col).sum()['Expansion Factor_h']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
New DistrictName,,
East Side,258650.11,340512.7994
Everett-Lynwood-Edmonds,122540.92,133795.0959
Kitsap,77365.82,79099.7800
North Seattle-Shoreline,122276.58,132703.7331
Renton-FedWay-Kent,215984.34,235851.5392
S.Kitsap,6998.18,14534.8730
Seattle CBD,221391.87,279329.9210
South Pierce,104619.23,122153.5370
Suburban Snohomish,76139.54,85395.7301


In [79]:
# Distribution
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

,2006 Survey,2014 Survey
New DistrictName,,
East Side,0.185130,0.205397
Everett-Lynwood-Edmonds,0.087709,0.080705
Kitsap,0.055375,0.047713
North Seattle-Shoreline,0.087520,0.080047
Renton-FedWay-Kent,0.154592,0.142265
S.Kitsap,0.005009,0.008767
Seattle CBD,0.158462,0.168491
South Pierce,0.074882,0.073683
Suburban Snohomish,0.054497,0.051511


In [80]:
display_charts(df_new, kind='bar', title='Job Location Distribution by County')

---

** Distribution of Worker Types**

In [ ]:
# Distribution
df_dist = pd.DataFrame([df[scen_name]/person_scen.count()['personID'],
                   df[base_name]/person_base.count()['personID']]).T

In [ ]:
display_charts(df_dist, kind='bar', title='Worker Type Distribution')

---

## School Location

In [96]:
# Join school geography
hh_per_scen_school_geog = pd.merge(hh_per_scen, taz_geog, left_on='School TAZ', right_on='TAZ')
hh_per_base_school_geog = pd.merge(hh_per_base, taz_geog, left_on='School TAZ', right_on='TAZ')

### By County

In [97]:
df = pd.DataFrame([hh_per_scen_school_geog.groupby('County').sum()['Expansion Factor_h'],
                   hh_per_base_school_geog.groupby('County').sum()['Expansion Factor_h']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
County,,
King,332074.17,692300.4446
Kitsap,47202.05,63941.3190
Pierce,141727.02,239051.0313
Snohomish,134688.11,189096.4190


In [98]:
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

,2006 Survey,2014 Survey
County,,
King,0.506449,0.584521
Kitsap,0.071988,0.053987
Pierce,0.216149,0.201835
Snohomish,0.205414,0.159657


In [99]:
display_charts(df_new, kind='bar', title='School Location Distribution by County')

### School Location by District

In [100]:
district_col = 'New DistrictName'
df = pd.DataFrame([hh_per_scen_school_geog.groupby(district_col).sum()['Expansion Factor_h'],
                   hh_per_base_school_geog.groupby(district_col).sum()['Expansion Factor_h']]).T
df.columns=([scen_name,base_name])
df

,2006 Survey,2014 Survey
New DistrictName,,
East Side,122008.20,239750.8870
Everett-Lynwood-Edmonds,63164.21,90176.3100
Kitsap,47202.05,63941.3190
North Seattle-Shoreline,63394.92,154168.0960
Renton-FedWay-Kent,73083.84,136088.7625
S.Kitsap,11018.92,14506.3820
Seattle CBD,45578.22,104991.5922
South Pierce,85980.99,148555.8866
Suburban Snohomish,71523.90,98920.1090


In [101]:
df_new = pd.DataFrame([df[scen_name]/df[scen_name].sum(),
             df[base_name]/df[base_name].sum()]).T
df_new

,2006 Survey,2014 Survey
New DistrictName,,
East Side,0.186076,0.202426
Everett-Lynwood-Edmonds,0.096332,0.076137
Kitsap,0.071988,0.053987
North Seattle-Shoreline,0.096684,0.130167
Renton-FedWay-Kent,0.111461,0.114902
S.Kitsap,0.016805,0.012248
Seattle CBD,0.069512,0.088646
South Pierce,0.131130,0.125428
Suburban Snohomish,0.109082,0.083520


In [102]:
display_charts(df_new, kind='bar', title='School Location Distribution by District')